In [1]:
import os
import hashlib
from ice_encryption import convertFiles

In [2]:
def get_digest(file_path):
    h = hashlib.sha1()
    with open(file_path, 'rb') as file:
        while True:
            chunk = file.read(h.block_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest().upper()

In [3]:
STW = lambda x: r'Z:\STEAMWORKS\content\brainbread2\brainbread2_shared\brainbread2\{}'.format(x).replace('\\', '/')

In [4]:
template = '''"Checksums"
{
	"Scripts"
	{
%s1
	}
	"Maps"
	{
%s2
	}
	"Tags"
	{
		"developer tags" "http://infestus-game.com/hl2world/bb2data/developers.txt"
		"donator tags" "http://infestus-game.com/hl2world/bb2data/donators.txt"
		"tester tags" "http://infestus-game.com/hl2world/bb2data/testers.txt"
		"ban data" "http://infestus-game.com/hl2world/bb2data/bans.txt"
		"blacklist data" "http://infestus-game.com/hl2world/bb2data/blacklistedservers.txt"
		//"disabled" "0"
	}
}'''
print(template)

"Checksums"
{
	"Scripts"
	{
%s1
	}
	"Maps"
	{
%s2
	}
	"Tags"
	{
		"developer tags" "http://infestus-game.com/hl2world/bb2data/developers.txt"
		"donator tags" "http://infestus-game.com/hl2world/bb2data/donators.txt"
		"tester tags" "http://infestus-game.com/hl2world/bb2data/testers.txt"
		"ban data" "http://infestus-game.com/hl2world/bb2data/bans.txt"
		"blacklist data" "http://infestus-game.com/hl2world/bb2data/blacklistedservers.txt"
		//"disabled" "0"
	}
}


In [5]:
data = {}
excluded = ['example_map', 'tutorial']

for path, _, files in os.walk(STW("maps"), topdown=True):
    for f in [x.lower() for x in files if x.lower().endswith('.bsp') and x.lower()[:-4] not in excluded]:
        data[f[:-4]] = os.path.getsize('{}/{}'.format(path, f)) if f.startswith('bbc_') or f.startswith('bba_') or f.startswith('bbs_') else 0

template = template.replace('%s2', '\n'.join(['\t\t\"{}\" \"{}\"'.format(k, v) for k, v in sorted(data.items())]))

In [6]:
files_to_check = [
    'data/game/game_base_player_arena.txt',
    'data/game/game_base_player_deathmatch.txt',
    'data/game/game_base_player_elimination.txt',
    'data/game/game_base_player_objective.txt',
    'data/game/game_base_player_powerups.txt',
    'data/game/game_base_player_shared.txt',
    'data/game/game_base_shared.txt',
    'data/game/gamemode_shared.txt',

    'data/npc/bandit.txt',
    'data/npc/bandit_leader.txt',
    'data/npc/bandit_johnson.txt',
    'data/npc/fred.txt',
    'data/npc/runner.txt',
    'data/npc/walker.txt',
    'data/npc/police.txt',
    'data/npc/swat.txt',
    'data/npc/riot.txt',
    'data/npc/military.txt',

    'scripts/weapon_beretta.txt',
    'scripts/weapon_deagle.txt',
    'scripts/weapon_akimbo_beretta.txt',
    'scripts/weapon_glock17.txt',
    'scripts/weapon_akimbo_glock17.txt',
    'scripts/weapon_ak47.txt',
    'scripts/weapon_g36c.txt',
    'scripts/weapon_remington700.txt',
    'scripts/weapon_remington.txt',
    'scripts/weapon_benelli_m4.txt',
    'scripts/weapon_winchester1894.txt',
    'scripts/weapon_sawedoff.txt',
    'scripts/weapon_akimbo_sawedoff.txt',
    'scripts/weapon_mac11.txt',
    'scripts/weapon_mp7.txt',
    'scripts/weapon_mp5.txt',
    'scripts/weapon_microuzi.txt',
    'scripts/weapon_famas.txt',
    'scripts/weapon_minigun.txt',
    'scripts/weapon_flamethrower.txt',
    'scripts/weapon_rex.txt',
    'scripts/weapon_akimbo_rex.txt',
    'scripts/weapon_zombhands.txt',
    'scripts/weapon_hands.txt',
    'scripts/weapon_machete.txt',
    'scripts/weapon_baseballbat.txt',
    'scripts/weapon_brick.txt',
    'scripts/weapon_fireaxe.txt',
    'scripts/weapon_sledgehammer.txt',
    'scripts/weapon_frag.txt',
    'scripts/weapon_propane.txt',
    'scripts/weapon_hatchet.txt',
    'scripts/weapon_m9_bayonet.txt', 
]

In [7]:
checksummed_files = []
for f in files_to_check:
    try:
        dig = get_digest(STW(f))
        checksummed_files.append('\t\t\"{}\" \"{}\"'.format(f, dig))
    except Exception as e:
        continue

template = template.replace('%s1', '\n'.join(checksummed_files))

In [8]:
with open('./checksums.txt', 'w') as f:
    f.write(template)

In [9]:
print(template)

"Checksums"
{
	"Scripts"
	{
		"data/game/game_base_player_arena.txt" "AC8F17D6BE544C06A99EBC6582BE4ACFEB024126"
		"data/game/game_base_player_deathmatch.txt" "C627255AB66E4C2D900E4FDF6FCD988E763A62EF"
		"data/game/game_base_player_elimination.txt" "8A83BA5EA07122F0301B8FD8112D288569926710"
		"data/game/game_base_player_objective.txt" "23DA5308F5B57158000FDFDD13ACAC865D263B38"
		"data/game/game_base_player_powerups.txt" "E0303A053F75E0E88864AA58195E99437204B8FC"
		"data/game/game_base_player_shared.txt" "CCB4FD30D7819995D1837803B035336708B10C63"
		"data/game/game_base_shared.txt" "07C2B9746AD2DEB2B4223AA3D87CCFE4A3CB0A92"
		"data/game/gamemode_shared.txt" "68E2041FEA60673F5C6A690732C49850A50148F2"
		"data/npc/bandit.txt" "76AB764A43BFFA77A6673FF2E0A14D64E2E87804"
		"data/npc/bandit_leader.txt" "42A04B94DA1A8078D62D58912262B03FA9D19A37"
		"data/npc/bandit_johnson.txt" "0B4B259C948918E0816B6629137B26F1905CBB0C"
		"data/npc/fred.txt" "0140D6BB6CA2465A1598A2E4509DA0A5BFC25F06"
		"data/npc/r

In [10]:
KEY = 'F3QxBzK6'
EXT1 = '.txt'
EXT2 = '.bbd'

In [11]:
convertFiles('./', './', EXT1, EXT2, KEY) # Encrypt

In [3]:
#convertFiles('./', './', EXT2, EXT1, KEY, True) # Decrypt